*This file will be a testbed for the combination of the Price and Yield calculations into the actual amortisation recalculations required by the application.*

In [1]:
# firstly, import the libraries that will be required
import pandas as pd
import yield_price_calcs as ypc

In [2]:
# next import the static data and trade data into their respective dataframes
static_df = pd.read_excel(".\Test Data\Test_data_xlsx.xlsx", "Static", index_col=0)
trades_df = pd.read_excel(".\Test Data\Test_data_xlsx.xlsx", "Trades", index_col=0)

In [3]:
# checking the data has loaded in correctly
static_df

,SEDOL,Fund,Maturity Date,PAR Value,Coupon Rate,Frequency,Accrual Method,Days in Month,Days in Year,Bond Type
Security Code,,,,,,,,,,
TESTSEC,9999999,TEST,2044-07-04,100,0.025,1,1,Actual,Actual,Bullet


In [4]:
trades_df

,Trade Type,Security Code,SEDOL,Contract Date,Settlement Date,Units,Unit Price (Local),Cost (Local)
Fund,,,,,,,,
TEST,PURCHASE,TESTSEC,9999999,2013-12-16,2013-12-19,70000,96.2900,67403.00
TEST,SALE,TESTSEC,9999999,2014-02-03,2014-02-06,30000,100.3030,28887.00
TEST,PURCHASE,TESTSEC,9999999,2014-11-10,2014-11-12,70000,118.0530,82637.10
TEST,SALE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,121153.10
TEST,PURCHASE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,140382.33
TEST,PURCHASE,TESTSEC,9999999,2015-02-27,2015-03-03,10000,139.7750,13977.50
TEST,PURCHASE,TESTSEC,9999999,2015-07-23,2015-07-27,10000,125.8780,12587.80
TEST,SALE,TESTSEC,9999999,2015-12-04,2015-12-08,20000,124.2940,25684.25
TEST,SALE,TESTSEC,9999999,2016-02-25,2016-02-29,110000,142.5720,141263.38


In [5]:
# allocating the static data required for the calculations from the static data dataframe to the variables
mat_date = static_df.loc["TESTSEC", "Maturity Date"]
red_value = static_df.loc["TESTSEC", "PAR Value"]
coupon = static_df.loc["TESTSEC", "Coupon Rate"]
coup_freq = static_df.loc["TESTSEC", "Frequency"]
acc_basis = static_df.loc["TESTSEC", "Accrual Method"]

In [6]:
# checking the values have been allocated as expected
print("Maturity Date: ", mat_date)
print("Redemption Value: ", red_value)
print("Coupon Rate: ", coupon)
print("Coupon Frequency:", coup_freq)
print("Accrual Basis: ", acc_basis)

Maturity Date:  2044-07-04 00:00:00
Redemption Value:  100
Coupon Rate:  0.025
Coupon Frequency: 1
Accrual Basis:  1


In [7]:
type(mat_date)

pandas._libs.tslibs.timestamps.Timestamp

In [8]:
mat_date.date()

datetime.date(2044, 7, 4)

In [9]:
# convert the Maturity Date column from a timestamp to a datetime object for ease of comparison
static_df["Maturity Date"]=static_df["Maturity Date"].apply(lambda x: x.date())

In [10]:
static_df

,SEDOL,Fund,Maturity Date,PAR Value,Coupon Rate,Frequency,Accrual Method,Days in Month,Days in Year,Bond Type
Security Code,,,,,,,,,,
TESTSEC,9999999,TEST,2044-07-04,100,0.025,1,1,Actual,Actual,Bullet


In [11]:
# check the conversion has worked
static_df.loc["TESTSEC", "Maturity Date"]

datetime.date(2044, 7, 4)

In [12]:
# convert the Contract Date and Settlement Date columns in the Trades dataframe into datetime objects
trades_df["Contract Date"]=trades_df["Contract Date"].apply(lambda x: x.date())
trades_df["Settlement Date"]=trades_df["Settlement Date"].apply(lambda x: x.date())

In [13]:
trades_df

,Trade Type,Security Code,SEDOL,Contract Date,Settlement Date,Units,Unit Price (Local),Cost (Local)
Fund,,,,,,,,
TEST,PURCHASE,TESTSEC,9999999,2013-12-16,2013-12-19,70000,96.2900,67403.00
TEST,SALE,TESTSEC,9999999,2014-02-03,2014-02-06,30000,100.3030,28887.00
TEST,PURCHASE,TESTSEC,9999999,2014-11-10,2014-11-12,70000,118.0530,82637.10
TEST,SALE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,121153.10
TEST,PURCHASE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,140382.33
TEST,PURCHASE,TESTSEC,9999999,2015-02-27,2015-03-03,10000,139.7750,13977.50
TEST,PURCHASE,TESTSEC,9999999,2015-07-23,2015-07-27,10000,125.8780,12587.80
TEST,SALE,TESTSEC,9999999,2015-12-04,2015-12-08,20000,124.2940,25684.25
TEST,SALE,TESTSEC,9999999,2016-02-25,2016-02-29,110000,142.5720,141263.38


Change of tack - test out the QuantLib library to see if the functionality in there can be used to simplify the calculations in any way

In [2]:
import quantlib as ql

ModuleNotFoundError: No module named 'quantlib'

In [3]:
!jupyter kernelspec list

Available kernels:
  python3    C:\Users\Stephen Parr\Anaconda3\share\jupyter\kernels\python3


In [4]:
import sys
sys.executable

'C:\\Users\\Stephen Parr\\Anaconda3\\python.exe'

In [5]:
sys.path

['C:\\Users\\Stephen Parr\\OneDrive\\Documents\\Python\\Amortisation\\VS Code',
 'C:\\Users\\Stephen Parr\\Anaconda3\\python37.zip',
 'C:\\Users\\Stephen Parr\\Anaconda3\\DLLs',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib',
 'C:\\Users\\Stephen Parr\\Anaconda3',
 '',
 'C:\\Users\\Stephen Parr\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Stephen Parr\\.ipython']

In [6]:
!conda list

# packages in environment at C:\Users\Stephen Parr\Anaconda3:
#
# Name                    Version                   Build  Channel
_anaconda_depends         2019.03                  py37_0  
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
alabaster                 0.7.12                   py37_0  
anaconda                  custom                   py37_1  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.12                   py37_0  
anaconda-project          0.8.4                      py_0  
argh                      0.26.2                   py37_0  
asn1crypto                1.3.0                    py37_0  
astroid                   2.3.3                    py37_0  
astropy                   4.0.1.post1      py37he774522_0  
atomicwrites              1.3.0                    py37_1  
attrs                     19.3.0                     py_0  
autopep8                  1.4.4                      py_0  
babel                     2.8

matplotlib-base           3.1.3            py37h64f37c6_0  
mccabe                    0.6.1                    py37_1  
menuinst                  1.4.16           py37he774522_0  
mistune                   0.8.4            py37he774522_0  
mkl                       2020.0                      166  
mkl-service               2.3.0            py37hb782905_0  
mkl_fft                   1.0.15           py37h14836fe_0  
mkl_random                1.1.0            py37h675688f_0  
mock                      4.0.1                      py_0  
more-itertools            8.2.0                      py_0  
mpmath                    1.1.0                    py37_0  
msgpack-python            1.0.0            py37h74a9793_1  
msys2-conda-epoch         20160418                      1  
multipledispatch          0.6.0                    py37_0  
navigator-updater         0.2.1                    py37_0  
nbconvert                 5.6.1                    py37_0  
nbformat                  5.0.4         

In [10]:
!conda env list

# conda environments:
#
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64
base                  *  C:\Users\Stephen Parr\Anaconda3
Python_Mega_Course       C:\Users\Stephen Parr\Anaconda3\envs\Python_Mega_Course
Python_Mega_Course_v2     C:\Users\Stephen Parr\Anaconda3\envs\Python_Mega_Course_v2



In [11]:
!jupyter kernelspec list

Available kernels:
  python3    C:\Users\Stephen Parr\Anaconda3\share\jupyter\kernels\python3


In [1]:
%conda install -c tonyroberts quantlib


Note: you may need to restart the kernel to use updated packages.


'C:\Users\Stephen' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import quantlib as ql

ModuleNotFoundError: No module named 'quantlib'

In [5]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [2]:
%conda install -c tonyroberts quantlib


Note: you may need to restart the kernel to use updated packages.


'C:\Users\Stephen' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import quantlib as ql

ModuleNotFoundError: No module named 'quantlib'

In [5]:
import sys
sys.executable

'C:\\Users\\Stephen Parr\\Anaconda3\\python.exe'

In [6]:
!conda env list

# conda environments:
#
                         C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64
base                  *  C:\Users\Stephen Parr\Anaconda3
Python_Mega_Course       C:\Users\Stephen Parr\Anaconda3\envs\Python_Mega_Course
Python_Mega_Course_v2     C:\Users\Stephen Parr\Anaconda3\envs\Python_Mega_Course_v2



In [7]:
!conda install --yes --prefix {sys.prefix} -c tonyroberts quantlib

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: quantlib


In [8]:
import quantlib as ql

ModuleNotFoundError: No module named 'quantlib'

In [11]:
sys.path

['C:\\Users\\Stephen Parr\\OneDrive\\Documents\\Python\\Amortisation\\VS Code',
 'C:\\Users\\Stephen Parr\\Anaconda3\\python37.zip',
 'C:\\Users\\Stephen Parr\\Anaconda3\\DLLs',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib',
 'C:\\Users\\Stephen Parr\\Anaconda3',
 '',
 'C:\\Users\\Stephen Parr\\AppData\\Roaming\\Python\\Python37\\site-packages',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\win32',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Stephen Parr\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Stephen Parr\\.ipython']

In [12]:
!jupyter kernelspec list

Available kernels:
  python3    C:\Users\Stephen Parr\Anaconda3\share\jupyter\kernels\python3


In [13]:
!conda install --yes --prefix /Users/"Stephen Parr"/Anaconda3 -c tonyroberts quantlib

Solving environment: ...working... done

# All requested packages already installed.



In [1]:
import quantlib

ModuleNotFoundError: No module named 'quantlib'

In [5]:
!conda list

# packages in environment at C:\Users\Stephen Parr\Anaconda3:
#
# Name                    Version                   Build  Channel
_anaconda_depends         2020.02                  py37_0  
_ipyw_jlab_nb_ext_conf    0.1.0                    py37_0  
alabaster                 0.7.12                   py37_0  
anaconda                  custom                   py37_1  
anaconda-client           1.7.2                    py37_0  
anaconda-navigator        1.9.12                   py37_0  
anaconda-project          0.8.4                      py_0  
argh                      0.26.2                   py37_0  
asn1crypto                1.3.0                    py37_0  
astroid                   2.3.3                    py37_0  
astropy                   4.0.1.post1      py37he774522_0  
atomicwrites              1.4.0                      py_0  
attrs                     19.3.0                     py_0  
autopep8                  1.4.4                      py_0  
babel                     2.8

patsy                     0.5.1                    py37_0  
pep8                      1.7.1                    py37_0  
pexpect                   4.8.0                    py37_0  
pickleshare               0.7.5                    py37_0  
pillow                    7.0.0            py37hcc1f983_0  
pip                       20.0.2                   pypi_0    pypi
pkginfo                   1.5.0.1                  py37_0  
pluggy                    0.13.1                   py37_0  
ply                       3.11                     py37_0  
powershell_shortcut       0.0.1                         2  
prettytable               0.7.2                    pypi_0    pypi
prometheus_client         0.7.1                      py_0  
prompt-toolkit            3.0.4                      py_0  
prompt_toolkit            3.0.4                         0  
psutil                    5.7.0            py37he774522_0  
psycopg2                  2.8.5            py37h26f1ce3_1    conda-forge
py             

In [4]:
import sys
!{sys.executable} -m pip install QuantLib-Python

'C:\Users\Stephen' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import quantlib as ql

ModuleNotFoundError: No module named 'quantlib'